In [119]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from config_vacation import g_key

In [120]:
#reading csv from part I
cities_data = '../WeatherPy/output_data/cities.csv'
cities_df = pd.read_csv(cities_data)

set_cities_df = cities_df.set_index('City_ID')
set_cities_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
City_ID,,,,,,,,,
0,fortuna,40.5982,-124.1573,55.00,94,75,21.85,US,1612139621
1,qaanaaq,77.4840,-69.3632,-4.81,88,100,1.48,GL,1612139486
2,vaini,-21.2000,-175.2000,80.60,89,90,10.36,TO,1612139621
3,broome,42.2506,-75.8330,24.01,86,90,9.22,US,1612139621
4,atuona,-9.8000,-139.0333,78.91,75,1,17.52,PF,1612139622
...,...,...,...,...,...,...,...,...,...
563,xichang,27.8964,102.2634,46.00,45,9,1.16,CN,1612139794
564,arvika,59.6553,12.5852,3.20,92,54,1.74,SE,1612139794
565,meadow lake,34.8014,-106.5436,50.00,27,40,3.89,US,1612139795


In [132]:
set_cities_df['Humidity'].max()

100

In [141]:
#humidity heatmap
gmaps.configure(api_key = g_key)

locations = set_cities_df[['Lat', 'Lng']]
fig = gmaps.figure(map_type= 'HYBRID')

humid = set_cities_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = False, max_intensity = set_cities_df['Humidity'].max(),
                                 point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [146]:
#ideal weather conditions
#max temp = 80, but nothing less than 60
#wind speed less than 10
#humidity nothing greater than 50
#cloudiness less than 25

#dropping temp
ideal_temp_index = set_cities_df[(set_cities_df['Max_Temp'] > 80) | (set_cities_df['Max_Temp'] < 50)].index
ideal_temp = set_cities_df.drop(ideal_temp_index, inplace = False)

#dropping wind
ideal_wind_index = ideal_temp[ideal_temp['Wind_Speed'] > 10].index
ideal_temp_wind = ideal_temp.drop(ideal_wind_index, inplace = False)

#dropping humidity
ideal_humid_index = ideal_temp_wind[ideal_temp_wind['Humidity'] > 30].index 
ideal_temp_wind_humidity = ideal_temp_wind.drop(ideal_humid_index, inplace = False)

#dropping cloudiness
ideal_cloud_index = ideal_temp_wind_humidity[ideal_temp_wind_humidity['Cloudiness'] > 35].index
ideal_weather_df = ideal_temp_wind_humidity.drop(ideal_cloud_index, inplace = False)

ideal_weather_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
City_ID,,,,,,,,,
101,san patricio,28.0170,-97.5169,73.40,23,1,8.05,US,1612139657
165,taoudenni,22.6783,-3.9836,67.93,23,0,8.72,ML,1612139675
167,banamba,13.5500,-7.4500,67.60,21,33,8.59,ML,1612139675
180,raymondville,26.4815,-97.7831,72.00,26,1,5.75,US,1612139678
191,tombouctou,20.0000,-3.0000,67.10,21,19,9.10,ML,1612139681
235,tignere,7.3667,12.6500,66.07,26,0,1.90,CM,1612139693
300,mankono,8.0586,-6.1897,75.45,25,0,4.18,CI,1612139716
309,valentin gomez farias,28.2167,-106.5833,61.00,28,6,5.99,MX,1612139719
350,madarounfa,13.3087,7.1560,67.86,22,27,9.31,NE,1612139731


In [123]:
#hotel df
hotel_df = ideal_weather_df
hotel_df['Hotel_Name'] = ""

In [124]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
City_ID,,,,,,,,,,
101,san patricio,28.0170,-97.5169,73.40,23,1,8.05,US,1612139657,
165,taoudenni,22.6783,-3.9836,67.93,23,0,8.72,ML,1612139675,
180,raymondville,26.4815,-97.7831,72.00,26,1,5.75,US,1612139678,
191,tombouctou,20.0000,-3.0000,67.10,21,19,9.10,ML,1612139681,
235,tignere,7.3667,12.6500,66.07,26,0,1.90,CM,1612139693,
300,mankono,8.0586,-6.1897,75.45,25,0,4.18,CI,1612139716,
309,valentin gomez farias,28.2167,-106.5833,61.00,28,6,5.99,MX,1612139719,
354,waw,7.7029,27.9953,72.63,21,0,6.76,SS,1612139732,
366,bara,10.3744,10.7288,70.77,23,0,7.67,NG,1612139735,


In [125]:
# geocoordinates
for i in range(len(hotel_df)):
    try:
        target_coordinates = f'{hotel_df.iloc[i,1]}, {hotel_df.iloc[i,2]}'
        params = {'location': target_coordinates,
                  'keyword': 'hotel',
                  'radius': 5000,
                  'key': g_key,}
        base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
        response = requests.get(base_url, params = params)
        hotel_data = response.json()
        print(hotel_data['results'][0]['name'])
        hotel_df.iloc[i, 9] = hotel_data['results'][0]['name']
    except:
        try: 
#wanted to add option of hostel in location where hotels may not be available
            target_coordinates = f'{hotel_df.iloc[i,1]}, {hotel_df.iloc[i,2]}'
            params = {'location': target_coordinates,
                      'keyword': 'hostel',
                      'radius': 5000,
                      'key': g_key,}
            base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
            response = requests.get(base_url, params = params)
            hostel_data = response.json()
            hotel_df.iloc[i, 9] = hostel_data['results'][0]['name']
        except:
            print('No hotel')
            hotel_df.iloc[i, 9] = 'No Hotel Found'


Americas Best Value Inn Sinton
No hotel
Texas Inn & Suites Raymondville
No hotel
Hotel Famanou
Hotel Cajoux
No hotel
Amarula Lodge and Restaurant
No hotel
Hotel le Consulat/Camping Touristique
Hotel Poufiret
AUN Hotel
Gitam Guest House
Viglosam Lodge


In [126]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
City_ID,,,,,,,,,,
101,san patricio,28.0170,-97.5169,73.40,23,1,8.05,US,1612139657,Americas Best Value Inn Sinton
165,taoudenni,22.6783,-3.9836,67.93,23,0,8.72,ML,1612139675,No Hotel Found
180,raymondville,26.4815,-97.7831,72.00,26,1,5.75,US,1612139678,Texas Inn & Suites Raymondville
191,tombouctou,20.0000,-3.0000,67.10,21,19,9.10,ML,1612139681,No Hotel Found
235,tignere,7.3667,12.6500,66.07,26,0,1.90,CM,1612139693,Hotel Famanou
300,mankono,8.0586,-6.1897,75.45,25,0,4.18,CI,1612139716,Hotel Cajoux
309,valentin gomez farias,28.2167,-106.5833,61.00,28,6,5.99,MX,1612139719,No Hotel Found
354,waw,7.7029,27.9953,72.63,21,0,6.76,SS,1612139732,Amarula Lodge and Restaurant
366,bara,10.3744,10.7288,70.77,23,0,7.67,NG,1612139735,No Hotel Found


In [155]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [158]:
marker_layer = gmaps.marker_layer(locations,
                                 info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))